# ETL_Part3

__!!!!!IMP!!!!: On the day when the timezone changes, we must implement a new logic. We utilize the current UTC time, but it's crucial not to mix data from before and after the time change. Otherwise, all data will advance by one hour. If you use the old UTC time, the data will be delayed by one hour.__

   _Portugal > Winter: "UTC +0" || Summers: "UTC +1" (+01:00 Paris now)_	

__Run the bellow 2 groups ( Libraries +USERNAME and DOWNLOAD FOLDER)__

 1 group will import the libraries, if some of it will not work, please open a new code cell and write: Example ( pip install shutill) and run it, this will install the library
 
 2 group will read you username and the folder where you have the downloads, you can have another then download, just ajust the variable " folder"

## Libraries

In [2]:
import os
import pandas as pd
import glob
import numpy as np
import time
import shutil
import re
import win32com.client
from pathlib import Path
from zipfile import ZipFile
from datetime import datetime, timedelta
import calendar
import win32com.client as win32
import subprocess
import warnings
import win32com.client as w3c
from openpyxl import load_workbook

warnings.filterwarnings('ignore')

--------------------------------------------------------------------------------------------------------------------------

# USERNAME and DOWNLOAD FOLDER

__!!!CHANGE THE  USER NAME, AND DOWNLOAD PATH IF NEEDED:__


In [3]:
#THE CODE SHOULD SHOW YOUR NAME:
username = os.getlogin()
print('Your username:'+ username)
#IF YOU DONT USE YOUR DOWNLOAD FOLDER CHANGE THE BELOW STRING TO YOUR FOLDER:
folder="Downloads"
downloads_folder_path = fr"C:\\Users\\{username}\\{folder}"
print('Folder where is your downloads:' + downloads_folder_path)
#Clean python cache
gen_py_path = fr"C:\Users\\{username}\\AppData\Local\Temp\gen_py"
shutil.rmtree(gen_py_path, ignore_errors=True)

date_today= datetime.today().strftime('%d-%m-%Y')


log_df = pd.DataFrame(columns=['Filename', 'Date', 'Status', 'Message'])
log_df.drop(log_df.index, inplace=True)
print("---------------------------------------------")
print(log_df)

Your username:bechara.9
Folder where is your downloads:C:\\Users\\bechara.9\\Downloads
---------------------------------------------
Empty DataFrame
Columns: [Filename, Date, Status, Message]
Index: []


--------------------------------------------------------------------------------------------------------------------------

# __________________________________MI DATA___________________

## Ads Review

Link:  https://datapower-va.bytelemon.com/bi/visit/6953507931698642950

- Dashboard: Ads & Account Review >>> Ads Review Raw Data


- You need to extract the current week ( files de Monday to Sunday)+ previous week data, on the fowlloing logic:

  - __Extract 2 files per week:__ 1º file with the filter (__AFFECT EXPERIENCIE : VALID TASK__) and other with  (__AFFECT EXPERIENCIE :INVALID TASK__)-- on the name file for the invalid, rename it by adding in the end the letter (v) "!! small letter !!"
  
  
- Filter to use:
  - __AUDITOR SITE:__  LIS_TP
  - __TASK FLAG:__ unselect both piscos on the bottom "Select all" +"Exclude"
  - __MANUAL OPERATION FLAG :__ 1  (Normally is all the time on 1)
  - __AFFECT EXPERIENCIE :__ valid task   or  invalid task 

!!Affter the download of all the data, run the below code and the code will merge together the files from the same week. !!

In [4]:
mi_ads_Ads_Review={'Ads_Review':r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\10. MI\Ads_Efficiency"}
def format_text(text, color_code, bold=False):
    return f"\033[1;{color_code}m{text}\033[0m" if bold else f"\033[{color_code}m{text}\033[0m"

for short_name, folder_path in mi_ads_Ads_Review.items():
    if os.path.isdir(folder_path) and (files := os.listdir(folder_path)):
        date_pattern = re.compile(r'_(\d{8})_(\d{8})')

        max_date_file = max(
            (file for file in files if (date_matches := date_pattern.search(file)) or (date_matches := re.search(r'(\d{8})_(\d{8})', file))),
            key=lambda file: datetime.strptime(date_matches.groups()[1], '%d%m%Y') if date_matches else datetime.min,
            default=None)

        if max_date_file:
            max_date_format = format_text(max_date_file, 31, bold=True)
            short_name_format = format_text(short_name, 31, bold=True)
            print(f"'{short_name_format}', the most recent file is: '{max_date_format}'")

'Ads_Review', the most recent file is: 'AdsReview_RawData-02092024_08092024.csv'


In [5]:
#fOLDER DESTINATION:
Ads_Review_destination_datadrops = r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\08 - data engineering\Data Drops\P.TTOK.CONT\MI_Ads"
Ads_Review_destination_EMEA = r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\10. MI\Ads_Efficiency"
#Sharepoitn
ADS_Review_onedrive = fr"C:\Users\\{username}\\Teleperformance\pt-dataanalytics-pbi-datadrops - P.TTOK.CONT\4. Raw Data and Aux Files\Ads_Efficiency"
#lOOK FOR THE FILE DOWNLOADED BASE ON ACONDITION:
Ads_Review_files = [filename for filename in os.listdir(downloads_folder_path) if 
                    filename.startswith("_Ads Review Raw Data") and not filename.endswith("v.csv")]
Ads_Error_files = [filename for filename in os.listdir(downloads_folder_path) if 
                   filename.startswith("_Ads Review Raw Data") and filename.endswith("v.csv")]
print(Ads_Review_files)
print(Ads_Error_files)

['_Ads Review Raw Data - last_week_valid.csv', '_Ads Review Raw Data - week_before_last_valid.csv']
['_Ads Review Raw Data - last_week_invalid_v.csv', '_Ads Review Raw Data - week_before_last_invalid_v.csv']


In [6]:
# Creation of empty dictionaries to store DataFrames
ads_review_dict = {}
ads_error_dict = {}

# For loop to read the files and create columns "Affect Experience"
for o in Ads_Review_files:
        p = pd.read_csv(downloads_folder_path + '\\' + o)
        p['Affect Experience'] = 'Valid Task'
        date = pd.to_datetime(p['operate_date'].min()).strftime('%d%m%Y')
        ads_review_dict.setdefault(date, []).append(p)
    
for q in Ads_Error_files:
        r = pd.read_csv(downloads_folder_path + '\\' + q)
        r['Affect Experience'] = 'Invalid Task'
        date = pd.to_datetime(r['operate_date'].min()).strftime('%d%m%Y')
        ads_error_dict.setdefault(date, []).append(r)
       
# Merge DataFrames with the same dates
merged_dfs = {}
for date, review_dfs in ads_review_dict.items():
    if date in ads_error_dict:
        error_dfs = ads_error_dict[date]
        merged_dfs[date] = pd.concat(review_dfs + error_dfs, ignore_index=True)

# Save the combined DataFrame to CSV files
for date, df in merged_dfs.items():
    try:
        start_date = pd.to_datetime(date, format='%d%m%Y')
        end_date = start_date + timedelta(days=6)
        start_date_str = start_date.strftime('%d%m%Y')
        end_date_str = end_date.strftime('%d%m%Y')
        filename = f'AdsReview_RawData-{start_date_str}_{end_date_str}.csv'
        df.to_csv(Ads_Review_destination_EMEA + "\\" + filename, index=False)
        df.to_csv(ADS_Review_onedrive + "\\" + filename, index=False)
        
        log_df = log_df.append({'Filename': filename, 'Date': date, 'Status': 'Success', 'Message': 'File saved successfully'}, ignore_index=True)
    except Exception as e:
        log_df = log_df.append({'Filename': filename, 'Date': date, 'Status': 'Error', 'Message': str(e)}, ignore_index=True)

# Display the log DataFrame
print(log_df)

                                  Filename      Date   Status  \
0  AdsReview_RawData-02092024_08092024.csv  02092024  Success   
1  AdsReview_RawData-26082024_01092024.csv  26082024  Success   

                   Message  
0  File saved successfully  
1  File saved successfully  


----------------------------------------END of this code Group--------------------------------------------------------------


## Labelling Project Details - Single Mod

link:https://datapower-va.bytelemon.com/bi/visit/6982209149375414277?immersive=1

- Go to "[Ops] Project Details" tab of the following dashboard: Select the tab __Single Moderation__				

- 1 file per month: Before day 15 extract previous month and current one.


- Filters:(select the following filters - which is also shown on the print attached)
    - __Top Filter__
        - __1 st Round resolve:__ >> Date Fixed ( select the full month that you need to extract)
        - __Project ID :__ unselect " Sellect all "
    - __Table__
        - __Date scale:__ resolve_date				
        - __Dynamic dimension:__ (remove "product_meego")	Click on Empty			
        - __Moderator info:__ Moderator (now with 审核员 )				
        - __"The second"-Dynamic dimension:__ department				
        - __Dynamic dimension:__ mode_name				
        - __Dynamic dimension:__ labeling_method	
        
Download using the three dots on the right side

    CSV UTF - 8
    Include a 0 on number of rows
                
!!AFTER DOWNLOAD RUN THE BELOW CODE!!

In [7]:
mi_folder = {'Mi Labeling': r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\10. MI\Labeling\Single Moderation"}

def format_text(text, color_code, bold=False):
    return f"\033[1;{color_code}m{text}\033[0m" if bold else f"\033[{color_code}m{text}\033[0m"

for short_name, folder_path in mi_folder.items():
    if os.path.isdir(folder_path) and (files := os.listdir(folder_path)):
        date_pattern = re.compile(r'_(\d{8})_(\d{8})')

        max_date_file = max(
            (file for file in files if (date_matches := date_pattern.search(file)) or (date_matches := re.search(r'(\d{8})_(\d{8})', file))),
            key=lambda file: datetime.strptime(date_matches.groups()[1], '%d%m%Y') if date_matches else datetime.min,
            default=None)

        if max_date_file:
            max_date_format = format_text(max_date_file, 31, bold=True)
            short_name_format = format_text(short_name, 31, bold=True)
            print(f"'{short_name_format}', the most recent file is: '{max_date_format}'")

'Mi Labeling', the most recent file is: '_Labelling Project Details - Single Mod - 01092024_30092024.csv'


# THIS CODE WILL READ THE LAST FILE ON THE mi LABELING FOLDER
mi_folder={'Mi Labeling': r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\10. MI\Labeling\Single Moderation"}
def format_text(text, color_code, bold=False):
    return f"\033[1;{color_code}m{text}\033[0m" if bold else f"\033[{color_code}m{text}\033[0m"

for short_name, folder_path in mi_folder.items():
    if os.path.isdir(folder_path) and (files := os.listdir(folder_path)):
        date_pattern = re.compile(r'_(\d{8})_(\d{8})')

        max_date_file = max(
            (file for file in files if (date_matches := date_pattern.search(file)) or (date_matches := re.search(r'(\d{8})_(\d{8})', file))),
            key=lambda file: datetime.strptime(date_matches.groups()[1], '%d%m%Y') if date_matches else datetime.min,
            default=None)

        if max_date_file:
            max_date_format = format_text(max_date_file, 31, bold=True)
            short_name_format = format_text(short_name, 31, bold=True)
            print(f"'{short_name_format}', the most recent file is: '{max_date_format}'")

# THIS CODE WILL DO ALL THE TRANFORMATION TO THE LABELING DATA

#WHERE TO SEND:
Labelling_project_destination_datadrops = r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\08 - data engineering\Data Drops\P.TTOK.CONT\MI_Labeling"
Labelling_project_destination_EMEA = r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\10. MI\Labeling\Single Moderation"

#Sharepoitn
Mi_Labeling_onedrive =fr"C:\Users\\{username}\\Teleperformance\pt-dataanalytics-pbi-datadrops - P.TTOK.CONT\4. Raw Data and Aux Files\Single Moderation"


#WILL SHOW THE NAME OF THE FILE SO YOU CAN HAVE SURE WE ARE TAKING THE RIGHT FILE:
Labelling_project_files = [filename for filename in os.listdir(downloads_folder_path) if 
                         filename.startswith("_Labelling Project")]
print(Labelling_project_files)


for w in Labelling_project_files:
 try:
    x = pd.read_csv(os.path.join(downloads_folder_path, w))
    x = x.rename(columns={"Moderator.1": "Moderator"})


     # Validação das colunas
    required_columns = ['resolve_date', 'department', 'mode_name', 'labeling_method']
    forbidden_columns = ['product_meego', 'department.1']

    # Verificar se todas as colunas necessárias estão presentes
    if not all(col in x.columns for col in required_columns):
        raise Exception("PS: The Validation Failed, Something is wrong - Required columns missing")

    # Verificar se as colunas proibidas estão presentes
    if any(col in x.columns for col in forbidden_columns):
        raise Exception("PS: The Validation Failed, Something is wrong - Forbidden columns present")

    
    start_date = pd.to_datetime(x.iloc[1:]['resolve_date'].min())
    start_date_year = start_date.year
    start_date_month = start_date.month
    
    final_day = calendar.monthrange(start_date_year, start_date_month)[1]
    len_start_date_month = str(start_date_month)
    if len(len_start_date_month) < 2:
        len_start_date_month = '0' + len_start_date_month    

    start_date_str = start_date.strftime('%d%m%Y')
    end_date_str = "%s%s%s" % (final_day, len_start_date_month, start_date_year)

    if len(start_date_str) < 8:
        start_date_str = '0' + start_date_str
    
    # Criação do nome do arquivo:
    new_filename = '_Labelling Project Details - Single Mod - %s_%s.csv' % (start_date_str, end_date_str)
     # Salvando o arquivo processado nas pastas especificadas
    datadrops_file_path = os.path.join(Labelling_project_destination_datadrops, new_filename)
    emea_file_path = os.path.join(Labelling_project_destination_EMEA, new_filename)
    ondrive_file_path = os.path.join(Mi_Labeling_onedrive, new_filename)

    x.to_csv(datadrops_file_path, index=False, encoding='utf_8_sig')
    x.to_csv(emea_file_path, index=False, encoding='utf_8_sig')
    x.to_csv(ondrive_file_path, index=False, encoding='utf_8_sig')


    # Atualizar o DataFrame de log em caso de sucesso
    log_df = log_df.append({'Filename': w, 'Date': start_date_str, 'Status': 'Success', 'Message': 'File saved successfully'}, ignore_index=True)
 except Exception as e:
        # Atualizar o DataFrame de log em caso de erro
    log_df = log_df.append({'Filename': w, 'Date': start_date_str, 'Status': 'Error', 'Message': str(e)}, ignore_index=True)

In [8]:
# THIS CODE WILL DO ALL THE TRANSFORMATION TO THE LABELING DATA

# WHERE TO SEND:
Labelling_project_destination_datadrops = r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\08 - data engineering\Data Drops\P.TTOK.CONT\MI_Labeling"
Labelling_project_destination_EMEA = r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\10. MI\Labeling\Single Moderation"

# Sharepoint
Mi_Labeling_onedrive = fr"C:\Users\\{username}\\Teleperformance\pt-dataanalytics-pbi-datadrops - P.TTOK.CONT\4. Raw Data and Aux Files\Single Moderation"

# WILL SHOW THE NAME OF THE FILE SO YOU CAN HAVE SURE WE ARE TAKING THE RIGHT FILE:
Labelling_project_files = [filename for filename in os.listdir(downloads_folder_path) if 
                         filename.startswith("_Labelling Project")]
print(Labelling_project_files)

for w in Labelling_project_files:
    try:
        x = pd.read_csv(os.path.join(downloads_folder_path, w))
        x = x.rename(columns={"Moderator.1": "Moderator"})

        # Validação das colunas
        required_columns = ['resolve_date', 'department', 'mode_name', 'labeling_method']
        forbidden_columns = ['product_meego', 'department.1']

        # Verificar se todas as colunas necessárias estão presentes
        if not all(col in x.columns for col in required_columns):
            raise Exception("PS: The Validation Failed, Something is wrong - Required columns missing")

        # Verificar se as colunas proibidas estão presentes
        if any(col in x.columns for col in forbidden_columns):
            raise Exception("PS: The Validation Failed, Something is wrong - Forbidden columns present")

        # Limpar espaços em branco nas células da coluna "department"
        x['department'] = x['department'].str.strip()

        # Validação do conteúdo da coluna "department"
        invalid_departments = x.loc[~x['department'].isin(['LIS-TP', '']), 'department'].unique()
        if len(invalid_departments) > 0:
            raise Exception(f"PS: The Validation Failed, Something is wrong - Invalid department values found: {invalid_departments}")

        start_date = pd.to_datetime(x.iloc[1:]['resolve_date'].min())
        start_date_year = start_date.year
        start_date_month = start_date.month

        final_day = calendar.monthrange(start_date_year, start_date_month)[1]
        len_start_date_month = str(start_date_month)
        if len(len_start_date_month) < 2:
            len_start_date_month = '0' + len_start_date_month    

        start_date_str = start_date.strftime('%d%m%Y')
        end_date_str = "%s%s%s" % (final_day, len_start_date_month, start_date_year)

        if len(start_date_str) < 8:
            start_date_str = '0' + start_date_str

        # Criação do nome do arquivo:
        new_filename = '_Labelling Project Details - Single Mod - %s_%s.csv' % (start_date_str, end_date_str)
        
        # Salvando o arquivo processado nas pastas especificadas
        datadrops_file_path = os.path.join(Labelling_project_destination_datadrops, new_filename)
        emea_file_path = os.path.join(Labelling_project_destination_EMEA, new_filename)
        ondrive_file_path = os.path.join(Mi_Labeling_onedrive, new_filename)

        x.to_csv(datadrops_file_path, index=False, encoding='utf_8_sig')
        x.to_csv(emea_file_path, index=False, encoding='utf_8_sig')
        x.to_csv(ondrive_file_path, index=False, encoding='utf_8_sig')

        # Atualizar o DataFrame de log em caso de sucesso
        log_df = log_df.append({'Filename': w, 'Date': start_date_str, 'Status': 'Success', 'Message': 'File saved successfully'}, ignore_index=True)
    except Exception as e:
        # Atualizar o DataFrame de log em caso de erro
        log_df = log_df.append({'Filename': w, 'Date': start_date_str, 'Status': 'Error', 'Message': str(e)}, ignore_index=True)


['_Labelling Project Details - Single Mod - 2024-09-10 08-10-16.csv', '_Labelling Project Details - Single Mod - 2024-09-10 08-10-54.csv']


In [9]:
date=datetime.today().strftime('%d-%m-%Y %H-%M-%S')
Log_file=log_df.to_excel(fr"C:\\Users\\{username}\\{folder}\\Logs_etl_part3_{date}.xlsx", index=False)

----------------------------------------END of this code Group--------------------------------------------------------------


--------------------------------------------------------------------------------------------------------------------------

## MI QA Appeals( MI ACCOUNT)


link: https://bytedance.sg.larkoffice.com/base/GHoXboG1YaWmigsDvKLlO8KzgFe?table=tblPELwHGA3cbJe5&view=veweOl34mL

__IMP:__  Use MI Account

__Tab:__ TAXONOMY: BS SR Qs performance

Simply click on the three dots, Export, CSV, Should say SR Lis, than click download and run the script

In [10]:
# New functionality to copy the file
filename = "TP-LIS - RCA - 2nd Round QA_BS SR Qs Performance_SR LIS.csv"
source_path = os.path.join(downloads_folder_path, filename)
destination_folder = r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\10. MI\Labeling QAs Appeals"
destination_path = os.path.join(destination_folder, filename)
#Sharepoitn
Labeling_QAS_Appeals_onedrive = fr"C:\Users\\{username}\\Teleperformance\pt-dataanalytics-pbi-datadrops - P.TTOK.CONT\4. Raw Data and Aux Files\Labeling QAs Appeals"


# Ensure the destination folder exists
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# Copy the file and print success message
try:
    shutil.copy(source_path, destination_path)
    shutil.copy(source_path, Labeling_QAS_Appeals_onedrive)
    
    print('File moved successfully!')
except FileNotFoundError:
    print(f'Error: File {filename} not found in {downloads_folder_path}')
except PermissionError:
    print('Error: Permission denied. Could not move the file.')
except Exception as e:
    print(f'Error: {str(e)}')

File moved successfully!


----------------------------------------END of this code Group--------------------------------------------------------------


--------------------------------------------------------------------------------------------------------------------------

## MI- ADS QA MODE ( MI ACCOUNT)


link: https://datapower-va.bytelemon.com/bi/visit/6953507931698642950?immersive=1

__IMP:__ You need to login to Datapower with the MI account, you need to logout to your main account and login again and selec the MI account.

__Tab:__  Dashboards __>__ All Dashboardsm __>__ Site Moderation Report __>__ QA from new dataset __>__Raw Data Export

__Table :__ QA Mode - RAW data - task level

__Filter:__ 

1: QA1 Date : Extract 2 files ( Current month + previous )

!!AFTER DOWNLOAD RUN THE BELOW CODES!!


In [11]:
#THIS CODE WILL READ THE LAST MI ADS QA MODE DATA FROM THE EMEA FOLDER
mi_ads_folder={'Mi ADS':r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\10. MI\Ads_Quality_QA_Mode"}


def format_text(text, color_code, bold=False):
    return f"\033[1;{color_code}m{text}\033[0m" if bold else f"\033[{color_code}m{text}\033[0m"

for short_name, folder_path in mi_ads_folder.items():
    if os.path.isdir(folder_path) and (files := os.listdir(folder_path)):
        date_pattern = re.compile(r'_(\d{8})_(\d{8})')

        max_date_file = max(
            (file for file in files if (date_matches := date_pattern.search(file)) or (date_matches := re.search(r'(\d{8})_(\d{8})', file))),
            key=lambda file: datetime.strptime(date_matches.groups()[1], '%Y%m%d') if date_matches else datetime.min,
            default=None)

        if max_date_file:
            max_date_format = format_text(max_date_file, 31, bold=True)
            short_name_format = format_text(short_name, 31, bold=True)
            print(f"'{short_name_format}', the most recent file is: '{max_date_format}'")

'Mi ADS', the most recent file is: '_QA Mode - RAW data - task level_20240901_20240930.csv'


In [12]:
#THIS CODE WILL VERIFY THE STRUTURE OF THE FILE AND VALIDATE IF THE ALL NEEDED COLUMNS ARE PRESENT ON THE DATA

#Folders destination:
MI_Ads_QA1_destination_EMEA =r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\10. MI\Ads_Quality_QA_Mode"

#Sharepoitn
Mi_QA_Mode_onedrive = fr"C:\Users\\{username}\\Teleperformance\pt-dataanalytics-pbi-datadrops - P.TTOK.CONT\4. Raw Data and Aux Files\Ads_Quality_QA_Mode"

#Location the file name base on a conditon:
MI_Ads_QA1_Files = [filename for filename in os.listdir(downloads_folder_path) if 
                    filename.startswith("_QA Mode - RAW data - task level -")]
MI_Ads_QA1_Files

expected_columns = ['QA1 Date', 'QA1 Date.1', 'qa1_inspect_id', 'moderator_name',
       'moderator_email', 'moderator_site', 'qa1_name', 'qa1_email',
       'qa1_site', 'task_mode', 'business_line', 'task_name',
       'moderation_stage', 'qa_country', 'QA1 task Link',
       'merged_task_inspect_status_pre_appeal', 'QA1 RCA Link',
       'qa1_appeal_result_task_level', 'QA1 Auditor Appeal Reason',
       'QA1 Appeal Response', 'QA1 Task Level RCA', 'qa1_rca_rca_result',
       'qa1_rca_rca_error', 'qa1_rca_rca_desc', 'qa1_result_type',
       'merged_task_inspect_status_post_appeal', 'row_reason_code_name',
       'QA2 Date', 'qa2_inspect_id', 'qa2_inspect_status_en',
       'qa2_rca_type_task_level', 'qa2_rca_rca_result', 'qa2_rca_rca_desc',
       'qa2_task_link', 'qa2_appeal_rca_task_link', 'qa2_name', 'qa2_email',
       'qa2_site', 'qa2_appeal_appeal_result',
       'qa2_appeal_original_appeal_reason'] 

validation_results = {}
def validate_csv_columns(csv_files, expected_columns):
    for csv_file in csv_files:
        try:
            # Read the CSV file
            df = pd.read_csv(csv_file)

            # Get the column names from the CSV file
            actual_columns = df.columns.tolist()

            # Check if all expected columns are present in the CSV file
            if set(expected_columns) == set(actual_columns):
                validation_results[csv_file] = True
            else:
                validation_results[csv_file] = False
        except FileNotFoundError:
            print(f"Error: CSV file '{csv_file}' not found.")
            validation_results[csv_file] = False
    return validation_results

# Check if all files passed the validation
if all(validation_results.values()):
    # Print "all good" in bold and green
    print("\033[1;32mAll good\033[0m")  # ANSI escape codes for bold (1) and green color (32)
else:
    # Print the validation results
    print("Validation results:")
    for csv_file, is_valid in validation_results.items():
        print(f"{csv_file}: {is_valid}")

All good


__If above is ALL GOOD run below cell, if error do not run the below cell. Ask MI DA about the error, to find to correct fix__

In [13]:
#For loop to read all files:
for i in MI_Ads_QA1_Files:
 try:
    #Read the files on the download folder:
    j = pd.read_csv(downloads_folder_path + '\\'+ i)
    j= j.rename(columns ={"QA1 Date.1":"QA1 Date"})
    #Date conditon to use on the filename:
    start_date = pd.to_datetime(j.iloc[:, 1].min())
    start_date_year = start_date.year
    start_date_month = start_date.month
    
    final_day = calendar.monthrange(start_date_year, start_date_month)[1]
    len_start_date_month = str(start_date_month)
    if len(len_start_date_month) < 2:
        len_start_date_month = '0'+ len_start_date_month    

    start_date = start_date.strftime('%Y%m%d')
    end_date3 = "%s%s%s" %(start_date_year, len_start_date_month, final_day)

    if len(start_date) < 8:
        start_date = '0'+ start_date     
    #Name creation base on a string +date above created
    k = '_QA Mode - RAW data - task level_%s_%s.csv' %(start_date, end_date3) 
    #Move the file with the new name:
    j.to_csv(MI_Ads_QA1_destination_EMEA + "\\" + k, index = False)
    j.to_csv(Mi_QA_Mode_onedrive + "\\" + k, index = False)

     # Atualizar o DataFrame de log em caso de sucesso
    log_df = log_df.append({'Filename': i, 'Date': start_date, 'Status': 'Success', 'Message': 'File saved successfully'}, ignore_index=True)
 except Exception as e:
        # Atualizar o DataFrame de log em caso de erro
    log_df = log_df.append({'Filename': i, 'Date': start_date, 'Status': 'Error', 'Message': str(e)}, ignore_index=True)

 
       

----------------------------------------END of this code Group--------------------------------------------------------------


--------------------------------------------------------------------------------------------------------------------------

--------------------------------------------------------------------------------------------------------------------------

# MI_Status

link: https://datapower-va.bytelemon.com/bi/visit/6953507931698642950

__TAB:__  Produtivity

__TAB:__  Raw DAta

__Tabela:__Statuses Raw Data


__Data time:__  3 files, current week + the last 2 weeks ( files de Monday to Sunday)

!!AFTER DOWNLOAD RUN THE BELOW CODE!!

In [14]:
#THIS CODE WILL VALIDATE THE LAST MI STATUS FILE FROM THE FOLDER:

mi_Statuses={'Mi Statuses':r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\10. MI\Ads_status"}
def format_text(text, color_code, bold=False):
    return f"\033[1;{color_code}m{text}\033[0m" if bold else f"\033[{color_code}m{text}\033[0m"

for short_name, folder_path in mi_Statuses.items():
    if os.path.isdir(folder_path) and (files := os.listdir(folder_path)):
        date_pattern = re.compile(r'_(\d{8})_(\d{8})')

        max_date_file = max(
            (file for file in files if (date_matches := date_pattern.search(file)) or (date_matches := re.search(r'(\d{8})_(\d{8})', file))),
            key=lambda file: datetime.strptime(date_matches.groups()[1], '%d%m%Y') if date_matches else datetime.min,
            default=None)

        if max_date_file:
            max_date_format = format_text(max_date_file, 31, bold=True)
            short_name_format = format_text(short_name, 31, bold=True)
            print(f"'{short_name_format}', the most recent file is: '{max_date_format}'")

'Mi Statuses', the most recent file is: 'Statuses Raw Data_02092024_08092024.csv'


In [15]:
#Code explain on MI- ADS QA1 code:
Statuses_destination_datadrops = r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\08 - data engineering\Data Drops\P.TTOK.CONT\MI_Ads_status"
Statuses_destination_EMEA = r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\10. MI\Ads_status"

#Sharepoitn
Mi_Ads_status_onedrive = fr"C:\Users\\{username}\\Teleperformance\pt-dataanalytics-pbi-datadrops - P.TTOK.CONT\4. Raw Data and Aux Files\Ads_status"

Statuses_files = [filename for filename in os.listdir(downloads_folder_path) if 
                  filename.startswith("_Statuses Raw")]
Statuses_files

['_Statuses Raw Data - 2024-09-10 08-14-40.csv',
 '_Statuses Raw Data - 2024-09-10 08-14-57.csv',
 '_Statuses Raw Data - 2024-09-10 08-15-13.csv']

In [16]:
#THE BELOW CODE WILL TRANSFOR ALL THE FILE OF MI STATUS AND SENT THEM TO THE FOLDER:

for jj in Statuses_files:
 try:
    u = pd.read_csv(downloads_folder_path + '\\'+ jj)
    u.drop(['language_skill_main'],axis=1, inplace=True)
    u = u.rename(columns ={"mod_role_extract":"mod_role","work_duty_extract":"work_duty"})
 
    start_date = pd.to_datetime(u['duty_date'].min())
    end_date = start_date + timedelta(days = 6)

    start_date = start_date.strftime('%d%m%Y')
    end_date = end_date.strftime('%d%m%Y')    
    
    if len(start_date) < 8:
        start_date = '0'+ start_date
    if len(end_date) < 8:
        end_date = '0'+ end_date        
       
    ff = 'Statuses Raw Data_%s_%s.csv' %(start_date, end_date)    
    
    u.to_csv(Statuses_destination_datadrops + '\\'+ ff, index = False)
    u.to_csv(Statuses_destination_EMEA + '\\'+ ff, index = False)
    u.to_csv(Mi_Ads_status_onedrive + '\\'+ ff, index = False)

    
 # Atualizar o DataFrame de log em caso de sucesso
    log_df = log_df.append({'Filename': jj, 'Date': start_date, 'Status': 'Success', 'Message': 'File saved successfully'}, ignore_index=True)
 except Exception as e:
        # Atualizar o DataFrame de log em caso de erro
    log_df = log_df.append({'Filename': jj, 'Date': start_date, 'Status': 'Error', 'Message': str(e)}, ignore_index=True)


----------------------------------------END of this code Group--------------------------------------------------------------


# MI_Produtivity

link: https://datapower-va.bytelemon.com/bi/visit/6953507931698642950?immersive=1

__TAB:__ Produtivity TAB /// Raw Data /// 

__Table:__ Productivity Raw Data // 2 files- Current +last week ( files de Monday to Sunday)


!!AFTER DOWNLOAD RUN THE BELOW CODE!!

In [17]:
#THIS CODE WILL CHECK THE LAST FILE PRESENT ON THE EMEA FOLDER:

mi_Productivity={'Mi Produtivity':r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\10. MI\Ads_productivity_v2"}
def format_text(text, color_code, bold=False):
    return f"\033[1;{color_code}m{text}\033[0m" if bold else f"\033[{color_code}m{text}\033[0m"

for short_name, folder_path in mi_Productivity.items():
    if os.path.isdir(folder_path) and (files := os.listdir(folder_path)):
        date_pattern = re.compile(r'_(\d{8})_(\d{8})')

        max_date_file = max(
            (file for file in files if (date_matches := date_pattern.search(file)) or (date_matches := re.search(r'(\d{8})_(\d{8})', file))),
            key=lambda file: datetime.strptime(date_matches.groups()[1], '%Y%m%d') if date_matches else datetime.min,
            default=None)

        if max_date_file:
            max_date_format = format_text(max_date_file, 31, bold=True)
            short_name_format = format_text(short_name, 31, bold=True)
            print(f"'{short_name_format}', the most recent file is: '{max_date_format}'")

'Mi Produtivity', the most recent file is: 'Productivity Raw Data_20240902_20240908.csv'


In [18]:
#Code explain on MI- ADS QA1 code:
Productivity_destination_EMEA = r'\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\10. MI\Ads_productivity_v2'

#Sharepoitn
Ads_productivity_onedrive = fr"C:\Users\\{username}\\Teleperformance\pt-dataanalytics-pbi-datadrops - P.TTOK.CONT\4. Raw Data and Aux Files\Ads_productivity_v2"

Productivity_files = [filename for filename in os.listdir(downloads_folder_path) if 
                  filename.startswith("_Productivity Raw Data")]
Productivity_files

['_Productivity Raw Data - 2024-09-10 08-16-08.csv',
 '_Productivity Raw Data - 2024-09-10 08-17-21.csv',
 '_Productivity Raw Data - 2024-09-10 08-17-39.csv']

In [19]:
#THE BELOW CODE WILL TRANSFOR ALL THE FILE OF MI PRODUTIVITY AND SENT THEM TO THE FOLDER:

for c in Productivity_files:
 try:
                      
    df = pd.read_csv(downloads_folder_path + '\\'+ c)[['p_date','auditor_staff_id','auditor_staff_name','mod_role_extract',
                                                      'delivery_country','task_type_en','queue_type','queue_name','platform_source',
                                                       'handling_time_min']]
    
    df = df.rename(columns ={"mod_role_extract":"mod_role","queue_type":"Queue Type","queue_name":"queue_name_en"})
    
 
    df['task_count']=1
    #Dates for the file name
    start_date = pd.to_datetime(df['p_date'].min())
    end_date = start_date + timedelta(days = 6)
    start_date = start_date.strftime('%Y%m%d')
    end_date = end_date.strftime('%Y%m%d')
    if len(start_date) < 8:
        start_date = '0'+ start_date
    if len(end_date) < 8:
        end_date = '0'+ end_date 
    #Groupby:
    df =df.groupby(['p_date','auditor_staff_id','auditor_staff_name','mod_role',
                                                      'delivery_country','task_type_en','Queue Type','queue_name_en','platform_source'],dropna=False).agg({
    'task_count':'sum',
    'handling_time_min':'sum'}).reset_index()
    dd = 'Productivity Raw Data_%s_%s.csv' %(start_date, end_date)    
    df.to_csv(Productivity_destination_EMEA + '\\'+ dd, index = False)
    df.to_csv(Ads_productivity_onedrive + '\\'+ dd, index = False)

    

     # Atualizar o DataFrame de log em caso de sucesso
    log_df = log_df.append({'Filename': c, 'Date': start_date, 'Status': 'Success', 'Message': 'File saved successfully'}, ignore_index=True)
 except Exception as e:
        # Atualizar o DataFrame de log em caso de erro
    log_df = log_df.append({'Filename': c, 'Date': start_date, 'Status': 'Error', 'Message': str(e)}, ignore_index=True)


# WELLNESS

 Download the file from the below link and open the file and Refresh all, and run the code: 


__File Link (2024):__  https://teleperformance-my.sharepoint.com/:x:/p/dijkstra_28_emea/ETb7JZ2sQ7RCsjFK6x6S7U4BVYn0WuL_eNkpaHbdnJ1VEw?email=pedro.esteves%40teleperformance.com&e=4%3AGTDzi5&fromShare=true&at=9&CID=93410ac1-2838-1df0-cac8-9b2ccfaedfb8



In [20]:
#File location:
#MANUAL WAY:
caminho_origem1 = fr"C:\Users\\{username}\\{folder}\Wellness Data 2024.xlsx"
caminho_origem1

# # location of 3 folders
Folder1 = r'\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\18. Wellness'
Folder2 = r'\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\08 - data engineering\Data Drops\P.TTOK.CONT\Wellness_Group'
Folder3 = r'\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\08 - data engineering\Data Drops\P.TTOK.CONT\Wellness_Individual'
Wellness_Onedrive= fr"C:\Users\\{username}\\Teleperformance\pt-dataanalytics-pbi-datadrops - P.TTOK.CONT\4. Raw Data and Aux Files\18. Wellness"

try:
    # Move the files to the folder
    shutil.copy(caminho_origem1, Folder1)
    log_df = log_df.append({'Filename': caminho_origem1, 'Date': date_today, 'Status': 'Success', 'Message': 'File copied successfully'}, ignore_index=True)
    shutil.copy(caminho_origem1, Folder2)
    log_df = log_df.append({'Filename': caminho_origem1, 'Date': date_today, 'Status': 'Success', 'Message': 'File copied successfully'}, ignore_index=True)
    shutil.copy(caminho_origem1, Folder3)
    log_df = log_df.append({'Filename': caminho_origem1, 'Date': date_today, 'Status': 'Success', 'Message': 'File copied successfully'}, ignore_index=True)
    shutil.copy(caminho_origem1, Wellness_Onedrive)
    log_df = log_df.append({'Filename': caminho_origem1, 'Date': date_today, 'Status': 'Success', 'Message': 'File copied successfully'}, ignore_index=True)

except Exception as e:
    log_df = log_df.append({'Filename': 'N/A', 'Date': date_today, 'Status': 'Error', 'Message': str(e)}, ignore_index=True)


-------------------------------END of this code Group--------------------------------------------------------------



## MMP Data 



__IMP:__ THIS NEEDS TO BE DONE WITH MI ACCOUNT AS WELL, THE SAME DAYS

__Link:__  https://byteworks-va.bytelemon.com/v2/workhour/correct

__Change Time Zone for Portugal, if Winter: "UTC", if Summers: "UTC +1" (+01:00 Paris now)__										

__- Filter:__ By time

__- Time range:__ Fixe Date - Day by Day ( Extract the current week and the previous week of data ), from 00:00 to 23:59				


__- Data aggregation method:__ By Employee												


__- Employee Status:__ Employee Status: Leave it empty, not selected, as the dashboard will pull the data with on and offbard data together.

__- Department FOR NOT MI ACCOUNT:__	TP_LIS 	/ TP_ADSO	

__- Department FOR MI ACCOUNT:__ MI_TP_LIS
															
											
__1- Select "Export Statistics" and download from \ 2-"My export" when the status is "completed"__	
													
__- NOTE:  We need to verify the size of the file, sometimes the file will download empty and without any data or with some data only.__														
																			



In [21]:
#Destination folder:
MMP_destination_datadrops = r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\08 - data engineering\Data Drops\P.TTOK.CONT\Aux_Codes"
MMP_destination_EMEA = r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\19. MMP Data Calibration - Aux Codes"
MMP_destination_desktop = r"C:\Users\bechara.9\Downloads"
Onedrive_destination= fr"C:\Users\\{username}\\Teleperformance\pt-dataanalytics-pbi-datadrops - P.TTOK.CONT\4. Raw Data and Aux Files\19. MMP Data Calibration - Aux Codes"


In [22]:
# Call the files and list them:
MMP_Data_files = [filename for filename in os.listdir(downloads_folder_path) if 
                  filename.startswith("data-")]
MMP_Data_files

['data-1725953307_1725953309755.csv',
 'data-1725953312_1725953314833.csv',
 'data-1725953318_1725953320830.csv',
 'data-1725953324_1725953326878.csv',
 'data-1725953330_1725953332667.csv',
 'data-1725953334_1725953337000.csv',
 'data-1725953338_1725953341370.csv',
 'data-1725953343_1725953346597.csv',
 'data-1725953349_1725953352101.csv',
 'data-1725953476_1725953488205.csv',
 'data-1725953484_1725953506203.csv',
 'data-1725953490_1725953508230.csv',
 'data-1725953497_1725953515600.csv',
 'data-1725953503_1725953526100.csv',
 'data-1725953512_1725953536855.csv',
 'data-1725953521_1725953546295.csv',
 'data-1725953536_1725953558880.csv',
 'data-1725953543_1725953565431.csv']

In [23]:
# Process each file
for i in MMP_Data_files:
    try:
        # Initialize flag
        ff = 0

        # Read the file and skip the first row
        dd = pd.read_csv(os.path.join(downloads_folder_path, i), skiprows=1)

        # Change the date to string
        file_date = str(pd.to_datetime(dd['日期/Date']).dt.strftime('%d%m%Y').astype(int).min())
        if len(file_date) < 8:
            file_date = '0' + file_date

        # Check if the department column contains any value starting with "MI_"
        if dd['部门/Department'].str.startswith('MI_').any():
            prefix = "MI_"
        else:
            prefix = ""

        # Create the filename with or without the MI_ prefix
        ff = prefix + 'MMP_Data_calibration_' + file_date + '.csv'

        # Save the file to the destination folder on the desktop
        new_file_path = os.path.join(MMP_destination_desktop, ff)
        dd.to_csv(new_file_path, index=False, encoding='utf_8_sig')

        # Copy the file to the other destinations
        shutil.copy(new_file_path, MMP_destination_datadrops)
        shutil.copy(new_file_path, MMP_destination_EMEA)
        shutil.copy(new_file_path, Onedrive_destination)
        

        # Log success
        log_df = log_df.append({'Filename': i, 'Date': file_date, 'Status': 'Success', 'Message': 'File copied successfully to all destinations'}, ignore_index=True)

    except Exception as e:
        # Log error
        log_df = log_df.append({'Filename': i, 'Date': file_date, 'Status': 'Error', 'Message': str(e)}, ignore_index=True)

# Optional: Remove the files from your download folder
#[os.remove(os.path.join(downloads_folder_path, filename)) for filename in os.listdir(downloads_folder_path) if filename.startswith("data-")]

# Display the log
print("---------------------------------------------")
print(log_df)

---------------------------------------------
                                             Filename        Date   Status  \
0             AdsReview_RawData-02092024_08092024.csv    02092024  Success   
1             AdsReview_RawData-26082024_01092024.csv    26082024  Success   
2   _Labelling Project Details - Single Mod - 2024...    01092024  Success   
3   _Labelling Project Details - Single Mod - 2024...    01082024  Success   
4   _QA Mode - RAW data - task level - 2024-09-10 ...    20240901  Success   
5   _QA Mode - RAW data - task level - 2024-09-10 ...    20240801  Success   
6        _Statuses Raw Data - 2024-09-10 08-14-40.csv    02092024  Success   
7        _Statuses Raw Data - 2024-09-10 08-14-57.csv         NaN    Error   
8        _Statuses Raw Data - 2024-09-10 08-15-13.csv    26082024  Success   
9    _Productivity Raw Data - 2024-09-10 08-16-08.csv    20240902  Success   
10   _Productivity Raw Data - 2024-09-10 08-17-21.csv         NaN    Error   
11   _Productivity

----------------------------------------END of this code Group--------------------------------------------------------------


# Coaching Survey



Just download the file:

link: https://teleperformance.larksuite.com/sheets/shtusIaUugKkg91iq4IA1YP2wgd

Format:  EXCEL


In [24]:
#File location:
Coaching_Survey = fr"C:\Users\\{username}\\{folder}\TikTok Internal Coaching Survey - Weekly(datarecord).xlsx"
#Destination:
Coaching_Survey_EMEA =r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\34. CoachingSurvey_Raw\CoachingSurvey_Raw.xlsx"
Coaching_Survey_DROPS=r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\08 - data engineering\Data Drops\P.TTOK.CONT\Coaching_Survey\CoachingSurvey_Raw.xlsx"
Coaching_destination= fr"C:\Users\\{username}\\Teleperformance\pt-dataanalytics-pbi-datadrops - P.TTOK.CONT\4. Raw Data and Aux Files\34. CoachingSurvey_Raw\CoachingSurvey_Raw.xlsx"

try:
    # Move the files to the EMEA folder
    shutil.copy(Coaching_Survey, Coaching_Survey_EMEA)
    log_df = log_df.append({'Filename': f'EMEA:{Coaching_Survey}', 'Date': date_today, 'Status': 'Success', 'Message': 'File copied successfully'}, ignore_index=True)
 
  
    # Move the files to the DROPS folder
    shutil.copy(Coaching_Survey, Coaching_Survey_DROPS)
    log_df = log_df.append({'Filename': f'Drops:{Coaching_Survey}', 'Date': date_today, 'Status': 'Success', 'Message': 'File copied successfully'}, ignore_index=True)
 

 # Move the files to the sharepoint folder
    shutil.copy(Coaching_Survey, Coaching_destination)
    log_df = log_df.append({'Filename': f'Drops:{Coaching_Survey}', 'Date': date_today, 'Status': 'Success', 'Message': 'File copied successfully'}, ignore_index=True)
 
except Exception as e:
    log_df = log_df.append({'Filename': Coaching_Survey, 'Date': date_today, 'Status': 'Error', 'Message': str(e)}, ignore_index=True)


-------------------------------END of this code Group--------------------------------------------------------------



--------------------------------------------------------------------------------------------------------------------------


# QA DATA ( ADSO : Ecolabeling Accuracy Individual) 

Go to the below link and go to the folders: ( ADSO), download the files  ( Ecolabeling Accuracy Individual ), if you need the file name, look on below paths location, there you have each file name.


link:https://teleperformance.sharepoint.com/:f:/r/sites/TikTok212/Shared%20Documents/Data%20Analytics/1.%20Quality%20files%20for%20DA%20Team%20PBI%20reports?csf=1&web=1&e=Pe8zSi


In [25]:

RS_ADSO = fr"C:\Users\\{username}\\{folder}\Ecolabeling Accuracy Individual.xlsx"


#EMEA

RS_ADSO_EMEA =r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\9. Quality Data\ADSO ECO Random Sampling"

#Data Drops
RS_ADSO_DROPS= r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\08 - data engineering\Data Drops\P.TTOK.CONT\Random_sampling_adso_post_appeal"

#Sharepoitn
RS_ADSO_onedrive = fr"C:\Users\\{username}\\Teleperformance\pt-dataanalytics-pbi-datadrops - P.TTOK.CONT\4. Raw Data and Aux Files\ADSO ECO Random Sampling"

# Struture Validation

In [26]:
import openpyxl

def verify_excel_files(files_info):
    for file_info in files_info:
        # Extract file path, sheet name, and required columns from file_info dictionary
        file_path = file_info["file_path"]
        sheet_name = file_info["sheet_name"]
        required_columns = file_info["required_columns"]
        
        try:
            # Load workbook in read-only mode
            workbook = openpyxl.load_workbook(file_path, read_only=True)
            # Check if the specified sheet exists in the workbook
            if sheet_name not in workbook.sheetnames:
                print(f"The sheet '{sheet_name}' is not present in the file '{file_path}'.")
                continue
            # Access the specified sheet
            sheet = workbook[sheet_name]
            # Retrieve header row and extract unique header values
            header_row = next(sheet.rows)
            header_values = {cell.value for cell in header_row}
            # Find missing columns in the header
            missing_columns = required_columns - header_values
            # Print appropriate message based on missing columns
            if missing_columns:
                print(f"The following columns are missing in the sheet '{sheet_name}' of file '{file_path}': {', '.join(missing_columns)}.")
            else:
                print(f"The sheet '{sheet_name}' of file '{file_path}' contains all required columns.")

        except FileNotFoundError:
            print(f"File '{file_path}' not found.")
        except Exception as e:
            print(f"An error occurred while processing file '{file_path}': {e}")

# Files to be read:
files_info = [
     
     {
        "file_path": fr"C:\Users\\{username}\\{folder}\Ecolabeling Accuracy Individual.xlsx",
        "sheet_name": "Sheet1",
        "required_columns": {'LOB','Week','Queue','Label','Labeler','TRUE_1','FALSE_1','Total Sampled','Accuracy','Date'}   
    }
]
# Call the function with the list of file information
verify_excel_files(files_info)


The sheet 'Sheet1' of file 'C:\Users\\bechara.9\\Downloads\Ecolabeling Accuracy Individual.xlsx' contains all required columns.


# Will move the files

In [27]:
try:
    # Move the files to the EMEA folder
    shutil.copy(RS_ADSO, RS_ADSO_EMEA)
    log_df = log_df.append({'Filename': f'EMEA:{RS_ADSO}', 'Date': date_today, 'Status': 'Success', 'Message': 'File copied successfully'}, ignore_index=True)
 
    
    # Move the files to the Drops folder
    shutil.copy(RS_ADSO, RS_ADSO_DROPS)
    log_df = log_df.append({'Filename': f'Drops:{RS_ADSO}', 'Date': date_today, 'Status': 'Success', 'Message': 'File copied successfully'}, ignore_index=True)
    
    # Move the files to the sharepoint folder
    shutil.copy(RS_ADSO, RS_ADSO_onedrive)
    log_df = log_df.append({'Filename': f'Drops:{RS_ADSO}', 'Date': date_today, 'Status': 'Success', 'Message': 'File copied successfully'}, ignore_index=True)
 


 
except Exception as e:
    log_df = log_df.append({'Filename': RS_ADSO, 'Date': date_today, 'Status': 'Error', 'Message': str(e)}, ignore_index=True)


---IF YOU DOWNLOAD LIVE R1 QUIZZ DATA RUN THE NEXT CODE----------------

--------------------------------------------------------------------------------------------------------------------------

# Live R1 Quizzes 

Go to the link and download the file Live R1 R3 - Quiz
Link:  https://teleperformance.sharepoint.com/sites/TikTok212/Shared%20Documents/Forms/AllItems.aspx?csf=1&web=1&e=Pe8zSi&cid=ccd55fb2%2D82d4%2D404d%2Db167%2Dc77620ab68a6&FolderCTID=0x012000B98B5AF80497F14392E44394E54AA19B&id=%2Fsites%2FTikTok212%2FShared%20Documents%2FData%20Analytics%2F1%2E%20Quality%20files%20for%20DA%20Team%20PBI%20reports%2FLIVE%20R1%2F2%2E%20LIVE%20R1%20quizzes&viewid=4d19ea0e%2D0774%2D4a0a%2D8fc4%2D204e3ee1184f

In [28]:
#FILE LOCATION:
xls = pd.ExcelFile(fr"C:\Users\\{username}\\{folder}\Live R1 R3 - Quiz.xlsx")


In [29]:
# Read each sheet:
df1 = pd.read_excel(xls, 'Live R1 FR')
df2 = pd.read_excel(xls, 'Live R1 SE')
df3 = pd.read_excel(xls, 'Live R1 IT')
df4 = pd.read_excel(xls, 'Live R1 NL')
df5 = pd.read_excel(xls, 'Live R1 HE')
df6 = pd.read_excel(xls, 'Live R2 IL')
df7 = pd.read_excel(xls, 'Live R2 SE')
df8 = pd.read_excel(xls, 'Gaming FR')
df9 = pd.read_excel(xls, 'Live R1 ES')

In [30]:
#concact all the sheets on a single file:
df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9])
#drop row vazias na coluna "Date"
df =df[~df.Date.isnull()]
# save the file on emea:
df.to_excel(r'\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\9. Quality Data\Quizzes\LIVE R1 & R3\Live R1 R3 - Quiz.xlsx', index=False)
#Drops folder location:
EMEA_location = r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\9. Quality Data\Quizzes\LIVE R1 & R3\Live R1 R3 - Quiz.xlsx"
Data_Drops= r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\08 - data engineering\Data Drops\P.TTOK.CONT\Quizzes_live _r1_r3"

#Sharepoitn
Quizzes_LIVE_onedrive = fr"C:\Users\\{username}\\Teleperformance\pt-dataanalytics-pbi-datadrops - P.TTOK.CONT\4. Raw Data and Aux Files\Quizzes_LIVE R1 & R3"

In [31]:
#move the file to Drops:
try:
    # Move the files from EMEA folder to DROPS folder 
    shutil.copy(EMEA_location, Data_Drops)
    log_df = log_df.append({'Filename': EMEA_location, 'Date': date_today, 'Status': 'Success', 'Message': 'File copied successfully'}, ignore_index=True)
    # Move the files from EMEA folder to Sharepoint folder 
    shutil.copy(EMEA_location, Quizzes_LIVE_onedrive)
    log_df = log_df.append({'Filename': EMEA_location, 'Date': date_today, 'Status': 'Success', 'Message': 'File copied successfully'}, ignore_index=True)
 
except Exception as e:
    log_df = log_df.append({'Filename': EMEA_location, 'Date': date_today, 'Status': 'Error', 'Message': str(e)}, ignore_index=True)


----------------------------------------END of this code Group--------------------------------------------------------------


--------------------------------------------------------------------------------------------------------------------------

# STD_DIM



Use the link to download the file and them run the code 


Link: https://teleperformance.sharepoint.com/:x:/r/sites/S.DAF.Operations_Data_Analytics/_layouts/15/Doc.aspx?sourcedoc=%7B239593F6-9215-440A-A196-777F73C4952C%7D&file=STD_DIMS_TTOK%20new%20(Teams%20Edit).xlsx&action=default&mobileredirect=true&cid=232a65da-64a

In [32]:
#File location:
STD_DIM=fr"C:\Users\\{username}\\{folder}\STD_DIMS_TTOK new (Teams Edit).xlsx"

#Folder destination:
EMEA_STD_DIM= r'\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\15. DIM files\STD_DIMS_TTOK new.xlsx'
Drop_DIM_LOB= r'\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\08 - data engineering\Data Drops\P.TTOK.CONT\Dim_LOB\STD_DIMS_TTOK new.xlsx'
Drop_Target= r'\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\08 - data engineering\Data Drops\P.TTOK.CONT\Dim_Target\STD_DIMS_TTOK new.xlsx'
Onedrive_STD_DIM=fr"C:\Users\{os.getenv('USERNAME')}\Teleperformance\pt-dataanalytics-pbi-datadrops - P.TTOK.CONT\4. Raw Data and Aux Files\15. DIM files\STD_DIMS_TTOK new.xlsx"
STD_DIM

'C:\\Users\\\\bechara.9\\\\Downloads\\STD_DIMS_TTOK new (Teams Edit).xlsx'

In [33]:
#THIS CODE WILL VALIDATE IS THE DOWNLOAD FILES HAVE ALL THE DATA THAT WE NEED 
# List for validation: 
STD_DIMv =pd.read_excel(fr"C:\Users\\{username}\\{folder}\STD_DIMS_TTOK new (Teams Edit).xlsx", sheet_name="DIM_LOB")
 
LobList = ['?', 'MI', 'ADSO', 'ALL LOB', 'AVIA', 'BFS', 'TTR1', 'CSAM', 'DSA',
       'LIVE R1', 'LIVE R2', 'LIVE R3', 'RESSO', 'STAFF', 'TTR2']
 
 
# Function that will check is the name of the above list is on the files:
def verifica_valores_na_coluna(coluna, LobList):
    new_values = []
    for valor_coluna in coluna:
        if valor_coluna.lower() not in [v.lower() for v in LobList]:
                 new_values.append(valor_coluna)
    if new_values:
            return "\033[1;31m!!!!!!FALSE!!!!!!\033[0m", new_values
    return "\033[1;32mTRUE\033[0m", []
 
# APPLY THIS CODE TO THE COLUMN OF OUR DATASET:
resultado_lob1, novos_valores = verifica_valores_na_coluna(STD_DIMv['LOB_L1'], LobList)
 
print("Validation Result_lob:", resultado_lob1)  # Accessing the first element of the tuple
print("New Values:", novos_valores)
 
 
#!!!!TRUE (ALL GOOD)
#!!!!FLASE(NOT GOOD=MISSING DATA ON OUR DOWNLOADS)

Validation Result_lob: TRUE
New Values: []


In [34]:
STD_DIMv2 =pd.read_excel(fr"C:\Users\\{username}\\{folder}\STD_DIMS_TTOK new (Teams Edit).xlsx", sheet_name="DIM_LOB")
 
sub_lob = ['?', 'ADS', 'ALL LOB', 'ASR', 'AVIA', 'BFS', 'CA', 'CF',
       'CLICKBAIT', 'COMMERCIAL CONTENT', 'CREATOR', 'CREATOR FUND',
       'CSAM', 'DSA', 'EDF', 'ENT', 'FIZZO', 'GAMING', 'GCL', 'HASH LAB',
       'HPCL', 'INSTRUCTIVE', 'LABELING', 'LCL', 'LEMON8',
       'LIVE LABELING', 'LIVE R1', 'LIVE R2', 'LIVE R3',
       'MENTAL HEALTH NARRATIVE', 'MI', 'NACL', 'OCR', 'ORIG', 'PML',
       'RESSO', 'SBP', 'SPS', 'SSM', 'STAFF', 'TAL', 'TRECO', 'TTR1',
       'TTR2', 'URL', 'VARL', 'VC', 'VDL', 'RVL', 'LTLM', 'CSL','HVC', 'CAPCUT','MCM','LVL','Issue Labeling','TCOR','VDL5.0', 'MULTI','SDS','COMMENTS QUALITY',
       'COMMENTS TAXONOMY','MARS', 'PHOTO LLM', 'STP', 'MASK Labeling']
 
 
# Function that will check is the name of the above list is on the files:
def verifica_valores_na_coluna1(coluna, sub_lob):
    new_values1 = []
    for valor_coluna in coluna:
        if valor_coluna.lower() not in [v.lower() for v in sub_lob]:
                 new_values1.append(valor_coluna)
    if new_values1:
            return "\033[1;31m!!!!!!FALSE!!!!!!\033[0m", new_values1
    return "\033[1;32mTRUE\033[0m", []


# APPLY THIS CODE TO THE COLUMN OF OUR DATASET:
resultado1, novos_valores1 = verifica_valores_na_coluna1(STD_DIMv2['LOB_L2'], sub_lob)
 
 
print("Validation Result:", resultado1)  # Accessing the first element of the tuple
print("New Values:", novos_valores1)
#!!!!TRUE (ALL GOOD)
#!!!!FLASE(NOT GOOD=MISSING DATA ON OUR DOWNLOADS)

Validation Result: !!!!!!FALSE!!!!!!
New Values: ['MARS', 'MARS', 'MARS', 'MARS', 'MARS', 'MARS', 'MARS', 'MARS', 'MARS', 'MARS', 'MARS', 'MARS', 'MARS', 'MARS', 'MARS', 'MARS', 'MARS', 'PHOTO LLM', 'STP', 'PHOTO LLM', 'PHOTO LLM', 'STP', 'STP', 'MARS', 'MARS', 'MARS', 'MARS', 'MARS', 'MARS', 'MASK Labeling', 'MASK Labeling']


In [35]:
#Move the file:
try:
    # Move the files from EMEA folder to DROPS folder 
    shutil.copy(STD_DIM, EMEA_STD_DIM)
    log_df = log_df.append({'Filename': f'Emea:{STD_DIM}', 'Date': date_today, 'Status': 'Success', 'Message': 'File copied successfully'}, ignore_index=True)
    shutil.copy(STD_DIM, Drop_DIM_LOB)
    log_df = log_df.append({'Filename': f'Drop:{STD_DIM}', 'Date': date_today, 'Status': 'Success', 'Message': 'File copied successfully'}, ignore_index=True)
    shutil.copy(STD_DIM, Drop_Target)
    log_df = log_df.append({'Filename': f'Drop:{STD_DIM}', 'Date': date_today, 'Status': 'Success', 'Message': 'File copied successfully'}, ignore_index=True)
    shutil.copy(STD_DIM, Onedrive_STD_DIM)
    log_df = log_df.append({'Filename': f'Drop:{STD_DIM}', 'Date': date_today, 'Status': 'Success', 'Message': 'File copied successfully'}, ignore_index=True)
except Exception as e:
    log_df = log_df.append({'Filename': STD_DIM, 'Date': date_today, 'Status': 'Error', 'Message': str(e)}, ignore_index=True)


# Log file creation

In [36]:
date=datetime.today().strftime('%d-%m-%Y %H-%M-%S')
Log_file=log_df.to_excel(fr"C:\\Users\\{username}\\{folder}\\Logs_etl_part3_{date}.xlsx", index=False)

----------------------------------------END of this code Group--------------------------------------------------------------
